In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import json
import shutil

# Path to your kaggle.json file
kaggle_json_path = '/content/drive/Othercomputers/My laptop/Current_Project_Code_Files/kaggle.json'

# Kaggle config directory
kaggle_config_dir = '/root/.config/kaggle'

# Create Kaggle config directory if it doesn't exist
os.makedirs(kaggle_config_dir, exist_ok=True)

# Copy the kaggle.json file to the Kaggle config directory
shutil.copy(kaggle_json_path, os.path.join(kaggle_config_dir, 'kaggle.json'))

# Set proper permissions for the kaggle.json file
os.chmod(os.path.join(kaggle_config_dir, 'kaggle.json'), 600)

# Now we can import and use the Kaggle API
import kaggle

# Download and unzip the dataset
kaggle.api.dataset_download_files('paultimothymooney/kermany2018', path='.', unzip=True)

print("Dataset downloaded successfully!")

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/kermany2018
Dataset downloaded successfully!


In [3]:
pip install imblearn

In [4]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

In [5]:
# Load the pre-trained InceptionV3 model without the top layers
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(299, 299, 3))

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

# Add new layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(4, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
def load_image_data(data_dir, class_names, img_size=(299, 299), max_images_per_class=None):
    images = []
    labels = []
    for class_index, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        image_count = 0
        for img_name in os.listdir(class_dir):
            if max_images_per_class and image_count >= max_images_per_class:
                break
            img_path = os.path.join(class_dir, img_name)
            img = load_img(img_path, target_size=img_size)
            img_array = img_to_array(img)
            img_array = img_array / 255.0
            images.append(img_array)
            labels.append(class_index)
            image_count += 1
    return np.array(images), np.array(labels)

data_dir = '/content/OCT2017 /train'
data_dir2 = '/content/OCT2017 /test'
class_names = ['NORMAL', 'CNV', 'DME', 'DRUSEN']

training_imgs, img_labels = load_image_data(data_dir, class_names, img_size=(299, 299), max_images_per_class=1000)
val_imgs, v_img_labels = load_image_data(data_dir2, class_names, img_size=(299, 299), max_images_per_class=40)

y_train = to_categorical(img_labels, num_classes=4)
y_val = to_categorical(v_img_labels, num_classes=4)

y_train_int = np.argmax(y_train, axis=1)

n_samples, height, width, channels = training_imgs.shape
X_reshaped = training_imgs.reshape((n_samples, -1))

oversample = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = oversample.fit_resample(X_reshaped, y_train_int)

X_resampled = X_resampled.reshape((-1, height, width, channels))
y_resampled = to_categorical(y_resampled, num_classes=4)

X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=42)

class_weights = compute_class_weight('balanced', classes=np.unique(img_labels), y=img_labels)
class_weight_dict = dict(enumerate(class_weights))

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint('best_inceptionv3_model.h5', save_best_only=True, monitor='val_accuracy', mode='max')
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_resampled,
    y_resampled,
    validation_data=(val_imgs, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[checkpoint, early_stop],
    class_weight=class_weight_dict
)

y_pred = model.predict(val_imgs)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val, axis=1)

report = classification_report(y_true, y_pred_classes, target_names=class_names)
print("\nClassification Report:")
print(report)

Epoch 1/20
125/125 [==============================] - 53s 389ms/step - loss: 0.8697 - accuracy: 0.6518 - val_loss: 0.3828 - val_accuracy: 0.8938
Epoch 2/20
125/125 [==============================] - 46s 368ms/step - loss: 0.6163 - accuracy: 0.7705 - val_loss: 0.3101 - val_accuracy: 0.8750
Epoch 3/20
125/125 [==============================] - 46s 365ms/step - loss: 0.5494 - accuracy: 0.7883 - val_loss: 0.3469 - val_accuracy: 0.8750
Epoch 4/20
125/125 [==============================] - 46s 369ms/step - loss: 0.4932 - accuracy: 0.8140 - val_loss: 0.3669 - val_accuracy: 0.8438
Epoch 5/20
125/125 [==============================] - 48s 385ms/step - loss: 0.4598 - accuracy: 0.8292 - val_loss: 0.2836 - val_accuracy: 0.9000
Epoch 6/20
125/125 [==============================] - 47s 378ms/step - loss: 0.4358 - accuracy: 0.8357 - val_loss: 0.2155 - val_accuracy: 0.9250
Epoch 7/20
125/125 [==============================] - 48s 387ms/step - loss: 0.3881 - accuracy: 0.8587 - val_loss: 0.1729 - val_ac